## 
# <h3><center>Heat Equation in 2D</center></h3>

$$
\frac{\partial u \left (x,\; t \right )} {\partial t} = D\left ( \frac{\partial^2 u \left (x,\; t \right )} {\partial x^2} + \frac{\partial^2 u \left (y,\; t \right )} {\partial y^2} \right )
$$
on the interval $x \in [0, Lx]$ and $y \in [0, Ly]$ with initial condition

$$
u(x, y, 0) = f (x,y),\:\: \forall  x \in [0, Lx]\:,\: y \in [0, Ly]
$$

#### Dirichlet boundary conditions

$$u(0,y, t) = c_1, \: u(L_x,y, t) = c_2 \quad \forall \, t \geq 0$$
$$u(x,0, t) = c_3, \: u(x,L_y, t) = c_4 \quad \forall \, t \geq 0$$


#### Von-Neumann boundary conditions

$$\frac{\partial u}{\partial x}\bigg|_{x=0} = -q_1 , \quad \frac{\partial u}{\partial x}\bigg|_{x=L_x} = q_2 \quad \forall \, t \geq0$$

$$\frac{\partial u}{\partial y}\bigg|_{x=0} = -q_3 , \quad  \frac{\partial u}{\partial y}\bigg|_{x=L_y} = q_4 \quad \forall \, t\geq 0$$


#### Robin boundary conditions

$$\frac{\partial u}{\partial x}\bigg|_{x=0} = \kappa_1 u(0, y, t), \quad \frac{\partial u}{\partial x}\bigg|_{x=L_x} = -\kappa_2 u(L_x, y, t) \quad \forall \, t \geq 0$$

$$\frac{\partial u}{\partial y}\bigg|_{x=0} = \kappa_1 u(x, 0, t), \quad \frac{\partial u}{\partial y}\bigg|_{x=L_y} = -\kappa_2 u(x, L_y, t) \quad \forall \, t \geq 0$$





#### Forward Euler

$$
\frac{u^{n+1}_{i,j}-u^n_{i,j}}{\Delta t} = D \left(\frac{u^{n}_{i+1,j} - 2u^n_{i,j} + u^n_{i-1,j}}{\Delta x^2}+\frac{u^{n}_{i,j+1} - 2u^n_{i,j} + u^n_{i,j-1}}{\Delta y^2}\right)
$$

$$
u^{n+1}_{i,j} = u^n_{i,j} + F_x\left(u^{n}_{i+1,j} - 2u^n_{i,j}  + u^n_{i-1,j}\right) + F_y\left(u^{n}_{i,j+1} - 2u^n_{i,j}  + u^n_{i,j-1}\right) \:, \:\:\: F_x = D\frac{\Delta t}{\Delta x^2} \:\:\text{and} \:\: F_y = D\frac{\Delta t}{\Delta y^2}
$$

#### Backward Euler

$$ 
\frac{u^{n}_i-u^{n-1}_i}{\Delta t} = D \left(\frac{u^{n}_{i+1,j} - 2u^n_{i,j} + u^n_{i-1,j}}{\Delta x^2}+\frac{u^{n}_{i,j+1} - 2u^n_{i,j} + u^n_{i,j-1}}{\Delta y^2}\right)
$$

$$  
- F_x u^n_{i-1,j} - F_y u^n_{i,j-1} + \left(2+  2F_x +  2F_y \right) u^{n}_{i,j} - F_x u^n_{i+1,j}   - F_y u^n_{i,j+1}= u_{i,j}^{n-1}
$$

$$ AU = b, \:\: U=(u^n_{1,1} ,\ldots,u^n_{N_x-1,N_y-1})$$


$$
\quad
$$


\begin{bmatrix}
2 + 2F_x + 2F_y & -F_x & -F_y & 0 & 0 \\
-F_x & 2 + 2F_x + 2F_y & -F_x & -F_y & 0 \\
-F_y & -F_x & 2 + 2F_x + 2F_y & -F_x & -F_y \\
0 & -F_y & -F_x & 2 + 2F_x + 2F_y & -F_x \\
0 & 0 & -F_y & -F_x & 2 + 2F_x + 2F_y \\
\end{bmatrix}



In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
plt.rcParams['animation.embed_limit'] = 1024

import scipy.sparse as sp
import scipy.sparse.linalg as spla

class DiffusionSolver2D:
    def __init__(self, Lx=1.0, Ly=1.0, T=5, D = 0.1, Nx=128, Ny = 128, Nt=256):
        
        # Parameters
        self.Lx = Lx  # Length of the x domain
        self.Ly = Ly  # Length of the domain
        self.T = T  # Total simulation time
        self.D = D  # Diffusion coefficient
        self.Nx = Nx  # Number of spatial x points
        self.Ny = Ny  # Number of spatial y points
        self.Nt = Nt  # Number of time steps

        # Initialize spatial and time points
        self.x = np.linspace(0, Lx, Nx)
        self.y = np.linspace(0, Ly, Ny)
        self.t = np.linspace(0, T, Nt)
        self.u = np.zeros((self.Nx, self.Ny, self.Nt))

        self.dx = np.diff(self.x)[0]  # Spatial x step |size
        self.dy = np.diff(self.y)[0]  # Spatial y step |size
        self.dt = np.diff(self.t)[0]  # Time step size

        self.Fx = self.D * self.dt / self.dx**2
        self.Fy = self.D * self.dt / self.dy**2

        # Initialize the quantity being diffused
        self.u[:,:,0] = self.initial_condition()
    
    def initial_condition(self):
        # Define the initial condition 
        X, Y = np.meshgrid(self.x, self.y)
        return np.sin(np.pi * X) * np.sin(np.pi * Y)

    def solve_FTCS(self, BC='Von-Neumann', BC_values = (0, 0, 0, 0)):
        # Solve the diffusion equation using the forward Euler method
        BC_top, BC_bottom, BC_left, BC_right = BC_values
        if BC == 'Dirichlet':
            for n in range(self.Nt - 1):
                self.u[1:-1,1:-1, n+1] = self.u[1:-1,1:-1, n] + \
                                         self.Fx*(self.u[2:,1:-1, n] - 2*self.u[1:-1,1:-1, n] + self.u[:-2,1:-1, n]) + \
                                         self.Fy*(self.u[1:-1,2:, n] - 2*self.u[1:-1,1:-1, n] + self.u[1:-1,:-2, n])
                
                self.u[0,:,:] = BC_left
                self.u[-1,:,:] = BC_right
                self.u[:,0,:] = BC_bottom
                self.u[:,-1,:] = BC_top
                

    
    def solve_BTCS(self, BC='Von-Neumann', BC_values = (0, 0, 0, 0)):
        # Solve the diffusion equation using the backward Euler method
        BC_top, BC_bottom, BC_left, BC_right = BC_values
        A = sp.diags([-self.Fx, 1+2*self.Fx, -self.Fx], [-1, 0, 1], shape=(self.Nx, self.Nx)).tolil()
        b = np.zeros((self.Nx,1))

        if BC == 'Dirichlet':
            A[0,0] = A[-1,-1] = 1
            A[0,1] = A[-1,-2] = 0
            A = A.tocsr()

            for n in range(self.Nt-1):
                b[:,0] = self.u[:,n]
                b[0,:] = BC_left
                b[-1,:] = BC_right
                self.u[:,n+1] = spla.spsolve(A,b)


    def solve_CN(self, BC='Von-Neumann', BC_left = 0, BC_right = 0):
        # Solve the diffusion equation using the Crank-Nicholson method
        A = sp.diags([-self.F/2, 1+self.F, -self.F/2], [-1, 0, 1], shape=(self.Nx, self.Nx)).tolil()
        B = sp.diags([self.F/2, 1-self.F, self.F/2], [-1, 0, 1], shape=(self.Nx, self.Nx)).tolil()
        b = np.zeros((self.Nx,1))

        if BC == 'Dirichlet':
            A[0,0] = A[-1,-1] = 1
            A[0,1] = A[-1,-2] = 0
            A = A.tocsr()
            B = B.tocsr()

            for n in range(self.Nt-1):
                b[:,0] = B @ self.u[:,n]
                b[0,:] = BC_left
                b[-1,:] = BC_right
                self.u[:,n+1] = spla.spsolve(A,b)


                
    def plot_animation(self):
        fig, ax = plt.subplots(figsize=(10, 7))
        
        # Initialize plot
        cax = ax.imshow(self.u[:, :, 0], origin='lower', extent=(0, self.Lx, 0, self.Ly),
                        cmap='inferno', vmin=self.u.min(), vmax=self.u.max())
        
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_title('Diffusion over time')
        
        # Add colorbar
        cbar = fig.colorbar(cax, ax=ax)
        cbar.set_label('u')
        
        # Define the update function for the animation
        def update(frame):
            cax.set_data(self.u[:, :, frame])
            ax.set_title(f'Diffusion over time - {frame * self.dt:.2f}s')
            return cax,
        
        ani = FuncAnimation(fig, update, frames=self.Nt, interval=50, blit=True)
        plt.close(ani._fig)
        
        return HTML(ani.to_jshtml())
